In [1]:
import json

import requests

headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:66.0) Gecko/20100101 Firefox/66.0"}

askreddit_url = "https://www.reddit.com/r/AskReddit"

In [2]:
def add_json_sufix(url):
    return url if url.endswith('.json') else url + '.json'

In [3]:
def get_with_headers(url):
    return requests.get(url, headers=headers)


def get_subreddit_posts(subreddit_url):
    print(f"Getting posts from {subreddit_url}...")
    subreddit_url = add_json_sufix(subreddit_url)
    response = get_with_headers(subreddit_url)
    raw_posts = response.json()['data']['children']

    posts = []
    for raw_post in raw_posts:
        post = {}
        raw_post = raw_post['data']
        post['title'] = raw_post['title']
        post['score'] = raw_post['score']
        post['url'] = raw_post['url']
        post['nsfw'] = raw_post['over_18']

        posts.append(post)

    posts = sorted(posts, key=lambda post: post['score'])
    return posts

In [4]:
def get_comments_from_post(post_url):
    post_url = add_json_sufix(post_url)

    print(f"Getting comments from {post_url}...")
    response = get_with_headers(post_url)
    raw_comments = response.json()[1]['data']['children']

    comments = []

    for raw_comment in raw_comments:
        comment = {}
        try:
            raw_comment = raw_comment['data']
            comment['score'] = raw_comment.get('score', 0)
            comment['content'] = raw_comment['body']
            comments.append(comment)
        except:
            pass
    comments = sorted(comments, key=lambda comment: comment['score'])
    if len(comments) > 5:
        comments = comments[:5]
    return comments


posts = sorted(get_subreddit_posts(askreddit_url), key=lambda post: post['score'])
for post in posts:
    post['comments'] = sorted(get_comments_from_post(post['url']), key=lambda comment: comment['score'])

with open('results.json', 'w') as output_file:
    json.dump(posts, output_file, sort_keys=True, indent=4)

Getting posts from https://www.reddit.com/r/AskReddit...
Getting comments from https://www.reddit.com/r/AskReddit/comments/c0sifa/substitute_teachers_of_reddit_whats_the_hardest/.json...
Getting comments from https://www.reddit.com/r/AskReddit/comments/c0rpcg/your_last_photo_is_how_you_die_how_do_you_die/.json...
Getting comments from https://www.reddit.com/r/AskReddit/comments/c0pshh/women_of_reddit_what_is_the_biggest_mistake_guys/.json...
Getting comments from https://www.reddit.com/r/AskReddit/comments/c0rjaa/what_is_the_worst_feeling_that_isnt_pain/.json...
Getting comments from https://www.reddit.com/r/AskReddit/comments/c0pt9o/serious_whats_something_harmless_your_parents_did/.json...
Getting comments from https://www.reddit.com/r/AskReddit/comments/c0ni9t/gamers_of_reddit_you_are_placed_into_an_arcade/.json...
Getting comments from https://www.reddit.com/r/AskReddit/comments/c0k8p6/what_do_you_get_complimented_for_the_most/.json...
Getting comments from https://www.reddit.com/r

In [5]:
print(posts)

[{'title': 'Substitute teachers of Reddit, what’s the hardest name you’ve ever had to pronounce?', 'score': 82, 'url': 'https://www.reddit.com/r/AskReddit/comments/c0sifa/substitute_teachers_of_reddit_whats_the_hardest/', 'nsfw': False, 'comments': [{'score': -7, 'content': "Lemonjello. Pronounced le-mon-je-lo. \nYou can imagine how I pronounced it. \n\nI haven't been so confused since the accident, and **my** name is hard to pronounce correctly."}, {'score': -7, 'content': 'So I’m not a substitute teacher but there was a girl back in my school named La-a but it was pronounced Ladasha.'}, {'score': -3, 'content': 'Shithead pronounced sha theed'}, {'score': -3, 'content': 'There\'s a "FWD: FWD: RE: For all my teachers" thread about a kid named: La-A\n\nPronounced: la-dash-uh'}, {'score': -2, 'content': 'My mother in law had a student named "jkmno" pronounced "No L"'}]}, {'title': 'Your last photo is how you die. How do you die?', 'score': 90, 'url': 'https://www.reddit.com/r/AskReddit/c

In [ ]:
def get_all_comments_from_subreddit(subreddit_url):
    posts = get_subreddit_posts(subreddit_url)
    all_comments = []
    for posts in posts:
        all_comments += get_comments_from_post(post['url'])